# Stream Cipher

In [1]:
from functools import reduce
from operator import xor

from streamcipher import LFSR, Bytes2Bool, Bool2Bytes
from streamcipher import berlekamp_massey, poly_int2str
from streamcipher import ShrinkingGenerator, SelfShrinkingGenerator
from streamcipher import frequency_test, block_frequency_test, runs_test

A stream cipher is a **symmetric key cipher** where the plaintext is encrypted (and ciphertext is decrypted) one digit at a time. A digit usually is either a *bit* or a *byte*. [[1]](#[1])

<img src="Images/StreamCipher.png" width="540" height="340" style="margin:auto"/>

Encryption (decryption) is achieved by xoring the plaintext (ciphertext) with a stream of pseudorandom digits obtained as an expansion of the key.

## LFSR

In an LFSR, the output from a standard shift register is fed back into its input causing an endless cycle. The feedback bit is the result of a linear combination of the shift register content and the polynomial coefficients. [[1]](#[1])

<img src="Images/LFSR.png" width="540" height="340" style="margin:auto"/>

From the block scheme:

\begin{flalign}
\left\{
\begin{array}{ll}
s_j[t] = s_{j+1}[t - 1], & j = 0,1,...,m - 2 \\
s_{m−1}[t] = \bigoplus_{j=0}^{m−1} p_{m−j}\times s_j[t − 1] \\
b[t] = s_{0}[t]
\end{array}
\right. &&
\end{flalign}

### testing

Test LFSR implementation (all attributes and mehtods) with the polynomial `poly = [4, 1, 0]`.

In [2]:
poly = [4,1,0]

# test instance with default state
test = LFSR(poly)
# print instance (__str__) with attributes
print(test)

# expected output
#
# degree 4
# poly   [1,0,0,1,1]
# state  b'\xf0'
# b      True
# fd     False

 degree          P(x)                state          b  fd 
----------------------------------------------------------
   4        x^4 + x^1 + 1      b'\xf0' -> 0b1111    1   0 


In [3]:
# set state
try:
    test.state = b'\x0a'
except ValueError:
    # expected to be here
    test.state = b'\xa0'
    print('Must keep attention to big endian, state set as', test.state)

Must keep attention to big endian, state set as b'\xa0'


In [4]:
# but not feedback or other attributes
try:
    test.output = False
except AttributeError:
    # expected to be here
    print('Cannot change output LFSR attribute')

Cannot change output LFSR attribute


In [5]:
# check of one cycle
state = b'@' # '\x40'

print('b_sequence:', test.cycle(state))
# expected out: [0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0]

b_sequence: [False, False, True, False, False, False, True, True, True, True, False, True, False, True, True, False]


In [6]:
# check of N steps
N = 6

spacing = '{:^5} {:^10} {:^3} {:^3}'
print(spacing.format('From','state','b','fd'))
print(spacing.format('    ','-------','---','---'))
print(spacing.format('    ',str(test.state),test.output,test.feedback))

print('\nAcross b_sequence:', test.run_steps(N), '\n')

print(spacing.format('To','state','b','fd'))
print(spacing.format('    ','-------','---','---'))
print(spacing.format('    ',str(test.state),test.output,test.feedback))
# expected out: 0 + [0, 1, 0, 0, 0, 1], s = b'\xf0', fd = False

From    state     b  fd 
       -------   --- ---
         b'@'     0   0 

Across b_sequence: [False, True, False, False, False, True] 

 To     state     b  fd 
       -------   --- ---
       b'\xf0'    1   0 


## Berlekamp Massey

Algorithm that will find the shortest linear-feedback shift register (LFSR) for a given binary output sequence, computing the minimal polynomial of a linearly recurrent sequence[[1]](#[1]). BM algorithm requir to don't have all-zero states to avoid to have only 0's sequence.

### testing

Test the implementation Berlekamp-Massy Algorithm with the sequence produced by the following LFSR:
- `poly = [3, 1, 0]` and `state = '\x60'`
- `poly = [5, 2, 1, 0]` and `state = '\x50'`
- `poly = [96, 81, 17, 15, 0]` and `state = b'streamcipher'`

In [7]:
##### poly = [3,1,0] and state = '\x60' ---> 0b011 #####

# how many bits of sequence? cycle, but at least 6
N = 6
# create LFSR and produce sequence
test = LFSR([3,1,0], b'\x60')
b_seq = test.run_steps(N)
# find shortest LFSR
poly, m = berlekamp_massey(b_seq)

print('The shortest polynomial is:', poly_int2str(poly))
print('Linear complexity:', m)

The shortest polynomial is: x^3 + x^1 + 1
Linear complexity: 3


In [8]:
##### poly = [5,2,1,0] and state = '\x50' ---> 0b01010 #####

# how many? at least 3 because seq is a repeating of 01
N = 3
# create instance of LFSR
test = LFSR([5,2,1,0], b'\x50')
b_seq = test.run_steps(N)
# find shortest LFSR
poly, m = berlekamp_massey(b_seq)

print('The shortest polynomial is: ', poly_int2str(poly))
print('Linear complexity:', m)

The shortest polynomial is:  x^2 + 1
Linear complexity: 2


In [9]:
##### poly = [96, 81, 17, 15, 0] and state = b'streamcipher' #####

# how many? at least 192
N = 192
# create instance of LFSR
test = LFSR([96,81,17,15,0], b'streamcipher')
b_seq = test.run_steps(N)
# find shortest LFSR
poly, m = berlekamp_massey(b_seq)

print('The shortest polynomial is: ', poly_int2str(poly))
print('Linear complexity:', m)

The shortest polynomial is:  x^96 + x^81 + x^17 + x^15 + 1
Linear complexity: 96


Load `'binary_sequence'` and find the polynomial corresponding to the shortest LFSR that can produce that sequence.

In [10]:
# load
with open(file='binary_sequence.bin', mode='rb') as f:
    b_seq = f.read()

# berlekamp_massey
poly, m = berlekamp_massey(Bytes2Bool(b_seq))

print('The shortest polynomial is: ', poly_int2str(poly))
print('Linear complexity:', m)

The shortest polynomial is:  x^16 + x^14 + x^13 + x^12 + x^9 + x^2 + 1
Linear complexity: 16


## LFSR-based Generator (Shrinking or Self-Shrinking Generator)

The shrinking generator comprises LFSRA that produces bits and LFSRS that selects the produced bits[[1]](#[1]).

<img src="Images/ShrinkingGenerator.png" width="540" height="340" style="margin:auto"/>

The Self-Shrinking Generator is composed by a single LFSR that produce bits and selects them depending on the value of the internal state[[1]](#[1]).

<img src="Images/Self-ShrinkingGenerator.png" width="540" height="340" style="margin:auto"/>

### testing

In [11]:
# find the generator assignment
group_name = b'CodeBreakers Union'
parity_bit = reduce(xor, Bytes2Bool(group_name))

if parity_bit:
    print('Assignment generator: Self-Shrinking Generator')
else:
    print('Assignment generator: Shrinking Generator')

Assignment generator: Shrinking Generator


Test Generator implementation (all attributes and methods).

In [12]:
# Try to encrypt a text
bytes_seq_plaintxt = b'Hello World!'
bool_seq_plaintxt = Bytes2Bool(bytes_seq_plaintxt)
# Shrinking with default state and polys
shrink_enc = ShrinkingGenerator()

# encryption
bool_ciphertxt = [bool_seq_plaintxt[0]^shrink_enc.output]
for b,s in zip(bool_seq_plaintxt[1:], shrink_enc):
    bool_ciphertxt.append(b^s)
    
ciphertxt = Bool2Bytes(bool_ciphertxt)

# print
print('Encryption of "' + bytes_seq_plaintxt.decode() + '" turns out to be', ciphertxt)

Encryption of "Hello World!" turns out to be b'\xb4suG\x01\x80\xa4K.\x89YV'


In [13]:
# Try to decrypt the text
bool_seq_ciphertxt = Bytes2Bool(ciphertxt)
# Shrinking with default state input
shrink_dec = ShrinkingGenerator([5,2,0], [3,1,0], b'\xf8', b'\xe0')

# decryption
bool_plaintxt = [bool_seq_ciphertxt[0]^shrink_dec.output]
for b,s in zip(bool_seq_ciphertxt[1:], shrink_dec):
    bool_plaintxt.append(b^s)

plaintxt = Bool2Bytes(bool_plaintxt)

# print
print('Plaintext is:', plaintxt.decode())

Plaintext is: Hello World!


Decrypt the ciphertext in `ciphertext_shrinking.bin` or `ciphertext_selfshrinking.bin`.

In [14]:
# for Shrinking Generator
polyA = [16, 15, 12, 10, 0]
polyS = [24, 11, 5, 2, 0]
stateA = b'\xc5\xd7' 
stateS = b'\x14\x84\xf8'

In [15]:
# for Self-Shrinking Generator
poly = [32, 16, 7, 2, 0]
selection_bit = 4
state = b'mJ\x9by'

In [16]:
# shrinking
shrink = ShrinkingGenerator(polyA, polyS, stateA, stateS)
# load
with open(file='ciphertext_shrinking.bin', mode='rb') as f:
    bytes_seq_chipertxt = f.read()

bool_seq_chipertxt = Bytes2Bool(bytes_seq_chipertxt)

bool_plaintxt = [bool_seq_chipertxt[0]^shrink.output]
for b,s in zip(bool_seq_chipertxt[1:], shrink):
    bool_plaintxt.append(b^s)

bytes_plaintxt = Bool2Bytes(bool_plaintxt)
print(bytes_plaintxt.decode())

The Shrinking Generator
Don Coppersmith, Hugo Krawczyk, Yishay Mansour
IBM T.J. Watson Research Center
Yorktown Heights, NY 10598

Abstract. We present a new construction of a pseudorandom generator based on a simple combination of two LFSRs. The construction bas attractive properties as simplicity (conceptual and implementation-wise), scalability (hardware and security), proven minimal security conditions (exponential period, exponential linear complexity, good statistical properties), and resistance to known attacks. The construction is suitable for practical implementation of efficient stream cipher cryptosystems.


##  Statistical Tests

Binary sequences must resemble a truly random sequence to be unpredictable. So,
<br><br>
**How to test randomness of a binary sequence?**
<br><br>
*NIST* proposes a statistical test suite. Each test assesses the presence of a specific pattern which would indicate that the sequence is not random. NIST Test Suite consists of 15 tests to assess the randomness of arbitrarily long binary sequences[[1]](#[1]):

* **The Frequency (Monobit) Test**
* **Frequency Test within a Block**
* **The Runs Test**
* Tests for the Longest-Run-of-Ones in a Block
* The Binary Matrix Rank Test
* The Discrete Fourier Transform (Spectral) Test
* The Non-overlapping Template Matching Test
* The Overlapping Template Matching Test
* Maurer's "Universal Statistical" Test
* The Linear Complexity Test
* The Serial Test
* The Approximate Entropy Test
* The Cumulative Sums (Cusums) Test
* The Random Excursions Test
* The Random Excursions Variant Test

A statistical test takes a binary sequence (and optionally other parameters) and returns a binary outcome PASS/FAIL. In general, a test consists of three phases[[1]](#[1]):
* Compute a statistic: any quantity computed from the binary sequence
* Compute the 𝒑-value: the probability to compute a statistic at least as extreme as the one actually computed, under the assumption that the binary sequence is random.
* Compare the 𝑝-value with the significance level of the test (𝛼): the probability of concluding that a truly random sequence is not random.

In [17]:
# create custom sequence
n = 4000
percentage_of_ones = 40
# compute number of ones in seq
num_ones = int(n * percentage_of_ones / 100)

custom_seq = [1 if i<num_ones else 0 for i in range(n)]

In [18]:
# create sequence from shrinking generator
n = 4000
shrink = ShrinkingGenerator()

shrink_seq = [int(shrink.output)]
for i,b in enumerate(shrink):
    shrink_seq.append(int(b))
    if i>n-3: break

### Frequency (Monobit) test

Determine whether the number of ones and zeros in a sequenceare approximately the same as would be expected for a truly random sequence.[[1]](#[1])

In [19]:
# frequency (monobit) test
custom_result = 'pass' if frequency_test(custom_seq) else 'fail'
shrink_result = 'pass' if frequency_test(shrink_seq) else 'fail'

print('Custom sequence', custom_result, 'frequency (monobit) test')
print('Generator sequence', shrink_result, 'frequency (monobit) test')

Custom sequence fail frequency (monobit) test
Generator sequence pass frequency (monobit) test


### Frequency test within a Block

Determine whether the frequency of ones in an 𝑀-bit block of a sequence is approximately 𝑀/2.[[1]](#[1])

In [20]:
# frequency block test
custom_result = 'pass' if block_frequency_test(custom_seq, 40) else 'fail'
shrink_result = 'pass' if block_frequency_test(shrink_seq, 40) else 'fail'

print('Custom sequence', custom_result, 'frequency (monobit) test')
print('Generator sequence', shrink_result, 'frequency (monobit) test')

Custom sequence fail frequency (monobit) test
Generator sequence pass frequency (monobit) test


### Runs test

determine whether the number of runs of ones and zeros in a sequence is as expected. A run of length 𝑘 is a sequence of 𝑘 identical bits bounded before and after with a bit of the opposite value.[[1]](#[1])

In [21]:
# runs test
custom_result = 'pass' if runs_test(custom_seq) else 'fail'
shrink_result = 'pass' if runs_test(shrink_seq) else 'fail'

print('Custom sequence', custom_result, 'frequency (monobit) test')
print('Generator sequence', shrink_result, 'frequency (monobit) test')

Custom sequence fail frequency (monobit) test
Generator sequence pass frequency (monobit) test


## Conclusion

To summarize, this research has extensively examined the complexities of stream ciphers and the fundamental components that drive their effectiveness, specifically emphasizing the significance of LFSRs and the Berlekamp-Massey Algorithm in enhancing cryptographic protection. By conducting a thorough evaluation of LFSR-based generators, including the Shrinking and Self-Shrinking Generators, we have investigated their ability to establish resilient encryption systems generating a passable tested truly random binary sequence.

## Reference

<span id='[1]' > [1] "<a href = "stream-cipher.pdf"> Stream Ciphers </a>"